## BERT 문장 벡터로 변환하기

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 2.8 MB 42.7 MB/s 
     |████████████████████████████████| 398 kB 48.8 MB/s 
     |████████████████████████████████| 57 kB 7.1 MB/s 
     |████████████████████████████████| 8.0 MB 56.1 MB/s 
     |████████████████████████████████| 806 kB 50.0 MB/s 
     |████████████████████████████████| 636 kB 49.6 MB/s 
     |████████████████████████████████| 829 kB 50.2 MB/s 
     |████████████████████████████████| 136 kB 59.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.1 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x3939e000 @  0x7fc48bbac615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |████████

In [2]:
from transformers import BertTokenizer

In [3]:
# Pretrained 모델을 사용할 때는 그 모델 훈련시킬 때 사용한 토크나이저를 그대로 사용해야 함
tokenizer = BertTokenizer.from_pretrained(
    "beomi/kcbert-base",
    do_lower_case = False
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

In [4]:
from transformers import BertConfig, BertModel

# 모델 설정 가져오기
pretrained_model_config = BertConfig.from_pretrained(
    "beomi/kcbert-base"
)

#모델 가져오기
model = BertModel.from_pretrained(
    "beomi/kcbert-base",
    config=pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
pretrained_model_config

BertConfig {
  "_name_or_path": "beomi/kcbert-base",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 300,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "transformers_version": "4.10.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30000
}

In [6]:
# 모델의 입력값
sentences = ["안녕하세요", "하이!"]

# 위에서 정의한 토크나이저로 워드피스 토큰화
features = tokenizer(
    sentences,
    max_length=10,
    padding="max_length",
    truncation=True,
)

In [7]:
features

{'input_ids': [[2, 19017, 8482, 3, 0, 0, 0, 0, 0, 0], [2, 15830, 5, 3, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0]]}

In [9]:
import torch
# feature 을 torch tensor로 변환
features = {k : torch.tensor(v) for k, v in features.items()}

In [11]:
# output은 모델의 여러 출력 결과를 한 곳에 묶은 것
outputs = model(**features)

In [12]:
# [] 안에 [[]], [[]] 로, 안녕하세요 와 하이 의 마지막 레이어 출력 벡터들을 담고 있다
outputs.last_hidden_state

tensor([[[-0.6969, -0.8248,  1.7512,  ..., -0.3732,  0.7399,  1.1907],
         [-1.4803, -0.4398,  0.9444,  ..., -0.7405, -0.0211,  1.3064],
         [-1.4299, -0.5033, -0.2069,  ...,  0.1285, -0.2611,  1.6057],
         ...,
         [-1.4406,  0.3431,  1.4043,  ..., -0.0565,  0.8450, -0.2170],
         [-1.3625, -0.2404,  1.1757,  ...,  0.8876, -0.1054,  0.0734],
         [-1.4244,  0.1518,  1.2920,  ...,  0.0245,  0.7572,  0.0080]],

        [[ 0.9371, -1.4749,  1.7351,  ..., -0.3426,  0.8050,  0.4031],
         [ 1.6095, -1.7269,  2.7936,  ...,  0.3100, -0.4787, -1.2491],
         [ 0.4861, -0.4569,  0.5712,  ..., -0.1769,  1.1253, -0.2756],
         ...,
         [ 1.2362, -0.6181,  2.0906,  ...,  1.3677,  0.8132, -0.2742],
         [ 0.5409, -0.9652,  1.6237,  ...,  1.2395,  0.9185,  0.1782],
         [ 1.9001, -0.5859,  3.0156,  ...,  1.4967,  0.1924, -0.4448]]],
       grad_fn=<NativeLayerNormBackward0>)

In [13]:
# 2개의 문장, 문장 길이는 10, 768차원의 벡터
outputs.last_hidden_state.shape

torch.Size([2, 10, 768])

In [14]:
# pooler_output은 BERT의 마지막 레이어 CLS 벡터에 행렬을 곱하고 tanh를 취한 결과(문장 임베딩) 다
outputs.pooler_output

tensor([[-0.1594,  0.0547,  0.1101,  ...,  0.2684,  0.1596, -0.9828],
        [-0.9221,  0.2969, -0.0110,  ...,  0.4291,  0.0311, -0.9955]],
       grad_fn=<TanhBackward0>)

+ last_hidden_state 은 두 문장을 단어 수준의 벡터 시퀀스로 변환한 것이고, pooler_output은 문장 수준의 벡터로 변환한 것임을 알 수 있다

+ last_hidden_State은 단어 수준 임베딩, pooler_output은 문장 수준 임베딩이라고 한다